In [1]:
!pip install  h3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 199.4 kB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/root/miniconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip install pandarallel

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 KB 203.1 kB/s eta 0:00:00a 0:00:01
  Created wheel for pandarallel: filename=pandarallel-1.6.2-py3-none-any.whl size=16411 sha256=edebde539fb2e76077960a63cef8a1d038b6c636bdd476f16d88b4e334b7c2dc
  Stored in directory: /root/.cache/pip/wheels/ae/c6/3c/534c2f44892fa82dd75f033b56fdd74db1c665f626363d319d
Successfully built pandarallel
You should consider upgrading via the '/root/miniconda3/bin/python -m pip install --upgrade pip' command.


In [3]:
import h3


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import requests
import os
from tqdm import tqdm
tqdm.pandas()



In [5]:
from pandarallel import pandarallel
os.environ['JOBLIB_TEMP_FOLDER'] = '/tmp'
pandarallel.initialize(nb_workers=40,progress_bar=True,verbose=2,use_memory_fs = False )

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [6]:
!env JOBLIB_TEMP_FOLDER=/tmp

MPLBACKEND=module://matplotlib_inline.backend_inline
HOSTNAME=k8s-master1.dmosk.local
LD_LIBRARY_PATH=/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64:/usr/include/x64_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
HOME=/root
PAGER=cat
CUDA_VERSION=10.1.243
NVIDIA_REQUIRE_CUDA=cuda>=10.1 brand=tesla,driver>=384,driver<385 brand=tesla,driver>=396,driver<397 brand=tesla,driver>=410,driver<411
NVIDIA_DRIVER_CAPABILITIES=compute,utility
TERM=xterm-color
PATH=/root/miniconda3/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
CUDA_PKG_VERSION=10-1=10.1.243-1
KMP_DUPLICATE_LIB_OK=True
KMP_INIT_AT_FORK=FALSE
LANG=C.UTF-8
JOBLIB_TEMP_FOLDER=/tmp
GIT_PAGER=cat
PWD=/root/work/Hakaton/champ-volg
CLICOLOR=1
JPY_PARENT_PID=7
PYDEVD_USE_FRAME_EVAL=NO
NVIDIA_VISIBLE_DEVICES=all


In [ ]:
data = pd.read_pickle("./data.pkl").reset_index(drop=True)
data=data.fillna('')

In [ ]:
data['type'].unique()

In [ ]:
data['type'].value_counts()[:50]

In [ ]:
data['name'].value_counts()[:50]

In [ ]:
data[['lat','lon']]=data['point'].str.strip('[]{} ').str.replace('"lat":','').str.replace('"lon":','').str.replace(' ','').str.split(',',n=2, expand=True)
data

In [ ]:
data['h3-4']=''
data['h3-5']=''
data['h3-6']=''
data['h3-7']=''
data['h3-8']=''
data['h3-9']=''
data['h3-10']=''


data['lat']=pd.to_numeric(data['lat'], downcast='float',errors = 'coerce').fillna(0)
data['lon']=pd.to_numeric(data['lon'], downcast='float',errors = 'coerce').fillna(0)




def hh3(i):
    #print(i)
    i['h3-4'] = h3.geo_to_h3(i['lat'], i['lon'], 4)
    i['h3-5'] = h3.geo_to_h3(i['lat'], i['lon'], 5)
    i['h3-6'] = h3.geo_to_h3(i['lat'], i['lon'], 6)
    i['h3-7'] = h3.geo_to_h3(i['lat'], i['lon'], 7)
    i['h3-8'] = h3.geo_to_h3(i['lat'], i['lon'], 8)
    i['h3-9'] = h3.geo_to_h3(i['lat'], i['lon'], 9)
    i['h3-10']= h3.geo_to_h3(i['lat'], i['lon'], 10)
    #print('----------')
    return i

data=data.parallel_apply(hh3,axis=1)



In [ ]:
data

In [ ]:
data.to_pickle('dataH3.pkl')